In [43]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import os
import glob
import re

In [2]:
df1 = pd.read_csv('3fingers_gzn/ego/3_a_ego.csv')
df1.head()

,x,y,time
0,242,376,1.727460e+09
1,241,373,1.727460e+09
2,240,369,1.727460e+09
3,240,362,1.727460e+09
4,242,358,1.727460e+09


In [44]:
def f1(df): # cos of initial
    x0 = df['x'][0]
    x2 = df['x'][2]
    y0 = df['y'][0]
    y2 = df['y'][2]
    
    x = x2 - x0
    y = y2 - y0
    
    return x / math.sqrt(x**2 + y**2 ) 

In [45]:
def f2(df): # sin of initial
    x0 = df['x'][0]
    x2 = df['x'][2]
    y0 = df['y'][0]
    y2 = df['y'][2]
    
    x = x2 - x0
    y = y2 - y0
    
    return y / math.sqrt(x**2 + y**2 ) 

In [46]:
def f3(df): # Length of Diagonal of Bounding Box
    max_x = df['x'].max()
    max_y = df['y'].max()
    
    min_x = df['x'].min()
    min_y = df['y'].min()
    
    return math.sqrt((max_y - min_y)**2 + (max_x - min_x)**2)

In [47]:
def f4(df): # Angle of Bounding Box
    max_x = df['x'].max()
    max_y = df['y'].max()
    
    min_x = df['x'].min()
    min_y = df['y'].min()
    
    if max_x - min_x == 0:
        return 1.571
    
    return math.atan((max_y - min_y) / (max_x - min_x))

In [48]:
def f5(df): # Distance between endpoints
    x0 = df['x'][0]
    y0 = df['y'][0]
    
    xlast = df['x'][len(df['x']) - 1]
    ylast = df['y'][len(df['y']) - 1]
    
    return math.sqrt((ylast - y0)**2 + (xlast - x0)**2)

In [49]:
def f6(df): # Sine of Endpoints Angle
    x0 = df['x'][0]
    y0 = df['y'][0]
    
    xlast = df['x'][len(df['x']) - 1]
    ylast = df['y'][len(df['y']) - 1]
    
    x = xlast - x0
    y = ylast - y0
    
    return x / math.sqrt(x**2 + y**2) 

In [50]:
def f7(df): # Cosine of Endpoints Angle
    x0 = df['x'][0]
    y0 = df['y'][0]
    
    xlast = df['x'][len(df['x']) - 1]
    ylast = df['y'][len(df['y']) - 1]
    
    x = xlast - x0
    y = ylast - y0
    
    return y / math.sqrt(x**2 + y**2 ) 

In [51]:
def f8(df): # Stroke Length
    res = 0
    for i in range(1, len(df['x']) - 1):
        res += math.sqrt((df['x'][i] - df['x'][i - 1]) ** 2 + (df['y'][i] - df['y'][i - 1]) ** 2)
        
    return res

In [52]:
def f9(df): # total relative rotation
    res = 0
    n = len(df)
    deltaxi = df['x'].diff()
    deltayi = df['delta_y'] = df['y'].diff()

    # Δx_(i-1) and Δy_(i-1)
    delta_x_prev = deltaxi.shift(1)
    delta_y_prev = deltayi.shift(1)
    
    angles = np.arctan2(
        deltaxi * delta_y_prev - delta_x_prev * deltayi,
        deltaxi * delta_x_prev + deltayi * delta_y_prev
    )
    
    return angles[2:n-1].sum()

In [53]:
def f10(df): # total absolute rotation
    res = 0
    n = len(df)
    deltaxi = df['x'].diff()
    deltayi = df['delta_y'] = df['y'].diff()

    # Δx_(i-1) and Δy_(i-1)
    delta_x_prev = deltaxi.shift(1)
    delta_y_prev = deltayi.shift(1)
    
    angles = np.arctan2(
        deltaxi * delta_y_prev - delta_x_prev * deltayi,
        deltaxi * delta_x_prev + deltayi * delta_y_prev
    )
    res = 0
    for i in angles[2:n-1]: 
        res += abs(i)
    
    return res

In [54]:
def f11(df): # total squared rotation
    res = 0
    n = len(df)
    deltaxi = df['x'].diff()
    deltayi = df['delta_y'] = df['y'].diff()

    # Δx_(i-1) and Δy_(i-1)
    delta_x_prev = deltaxi.shift(1)
    delta_y_prev = deltayi.shift(1)
    
    angles = np.arctan2(
        deltaxi * delta_y_prev - delta_x_prev * deltayi,
        deltaxi * delta_x_prev + deltayi * delta_y_prev
    )
    res = 0
    for i in angles[2:n-1]: 
        res += i ** 2
    
    return res

In [55]:
def f12(df): # maximum speed squared
    res = 0
    for i in range(1, len(df['x']) - 1):
        if df['time'][i] - df['time'][i - 1] == 0:
            continue
        temp = ((df['x'][i] - df['x'][i - 1]) ** 2 + (df['y'][i] - df['y'][i - 1]) ** 2) / (df['time'][i] - df['time'][i - 1]) ** 2
        res = max(res, temp)
    
    return res

In [56]:
def f13(df): # total duration
    return df['time'][len(df) - 1] - df['time'][0]

In [57]:
def f15(df): # stroke length ⁄ distance between endpoints
    res = f8(df) / f5(df)
    return res

In [71]:
# construct dataframe
result = {
    'sketch': [],
    'f01': [],
    'f02': [],
    'f03': [],
    'f04': [],
    'f05': [],
    'f06': [],
    'f07': [],
    'f08': [],
    'f09': [],
    'f10': [],
    'f11': [],
    'f12': [],
    'f13': [],
    'f15': []
}

# Create the DataFrame
result_df = pd.DataFrame(result)

In [72]:
main_folder_path = '3fingers_gzn'

subfolders = sorted([f for f in os.listdir(main_folder_path) if os.path.isdir(os.path.join(main_folder_path, f))])
def extract_number(filename):
    match = re.search(r'_(\d+)\.csv', filename)
    return int(match.group(1)) if match else float('inf')  # Use 'inf' for files without a number


for subfolder in subfolders:
    subfolder_path = os.path.join(main_folder_path, subfolder)
    
    # Check if the subfolder path is a directory
    
    if os.path.isdir(subfolder_path):
        # Get all CSV files in the subfolder
        
        csv_files = glob.glob(os.path.join(subfolder_path, '*.csv'))
        csv_files_sorted = sorted(csv_files, key=lambda x: extract_number(os.path.basename(x)))
        
#         print(subfolder_path)
        # Loop through the CSV files in the subfolder and read them
        for file in csv_files_sorted:
            name = os.path.splitext(os.path.basename(file))[0]
#             print(name)
            df = pd.read_csv(file)
            
        # add a new row
            new_row = {'sketch': name, 'f01': f1(df), 'f02': f2(df), 'f03': f3(df), 
                       'f04': f4(df), 'f05': f5(df), 'f06': f6(df), 'f07': f7(df),
                       'f08': f8(df), 'f09': f9(df), 'f10': f10(df), 'f11': f11(df),
                       'f12': f12(df), 'f13': f13(df),'f15': f15(df)}
                    
            result_df = pd.concat([result_df, pd.DataFrame([new_row])], ignore_index=True)

/var/folders/78/z0r8ycv11bx75tjy78t_lnsr0000gn/T/ipykernel_47211/2160737091.py:10: RuntimeWarning: invalid value encountered in true_divide
  return x / math.sqrt(x**2 + y**2 )


In [73]:
result_df.to_csv('3fingers_gzn_features.csv', index=False)